In [3]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
from shapely.wkt import loads
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon
from sqlalchemy import create_engine
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
import mplleaflet
import seaborn as sns

In [19]:
# Função para reconstruir a trajetória
def get_routes(conn):
    query = f"""
    SELECT
        linha,
        week_day,
        ST_AsText(start_point) as start_point,
        ST_AsText(end_point) as end_point,
        ST_AsText(route_line_out) as route_line_out,
        ST_AsText(route_line_back) as route_line_back
    FROM public.bus_routes
    """
    
    # Carregar dados em um GeoDataFrame
    df = pd.read_sql(query, conn)

    return df

# Função para reconstruir a trajetória
def get_bus_data(conn, linha):
    # Selecionar dados da tabela bus_routes para a linha específica
    query = f'''select 
                    *, 
                    ST_Transform(geom::geometry, 4326) AS geometry 
                from vehicle_tracking_filtered where 
                linha = '{linha}'
            '''
        
    # Carregar dados em um GeoDataFrame
    gdf = gpd.read_postgis(query, conn, geom_col='geometry', crs='EPSG:4326')

    return gdf

### Extract Trajectory and Bus_Data

In [20]:

# Conectar ao banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)



routes = get_routes(conn)
bus_df = get_bus_data(conn, '606')
conn.close()


C:\Users\lucas\AppData\Local\Temp\ipykernel_34060\1324893997.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [21]:
bus_df.sort_values(by='datahora')

,ordem,latitude,longitude,datahora_epoch,datahora,velocidade,linha,datahoraenvio_epoch,datahoraservidor_epoch,datahoraservidor,datahoraenvio,geom,geometry
0,B25565,-22.91091,-43.21163,1714012255,2024-04-24 23:30:55,54,606,1714014080,1714014098,2024-04-25 00:01:38,2024-04-25 00:01:20,0101000020E61000001E6D1CB1169B45C07099D36531E9...,POINT (-43.21163 -22.91091)
1,B25565,-22.91060,-43.20788,1714012285,2024-04-24 23:31:25,37,606,1714014080,1714014098,2024-04-25 00:01:38,2024-04-25 00:01:20,0101000020E61000000ABFD4CF9B9A45C0D3BCE3141DE9...,POINT (-43.20788 -22.91060)
2,B25565,-22.90811,-43.20950,1714012315,2024-04-24 23:31:55,32,606,1714014080,1714014098,2024-04-25 00:01:38,2024-04-25 00:01:20,0101000020E6100000894160E5D09A45C0AA2B9FE579E8...,POINT (-43.20950 -22.90811)
3,B25565,-22.90711,-43.20949,1714012345,2024-04-24 23:32:25,1,606,1714014080,1714014098,2024-04-25 00:01:38,2024-04-25 00:01:20,0101000020E6100000666B7D91D09A45C0E3DF675C38E8...,POINT (-43.20949 -22.90711)
4,B25565,-22.90511,-43.20950,1714012375,2024-04-24 23:32:55,37,606,1714014090,1714014098,2024-04-25 00:01:38,2024-04-25 00:01:30,0101000020E6100000894160E5D09A45C05648F949B5E7...,POINT (-43.20950 -22.90511)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
926586,B25535,-22.91254,-43.26709,1715216314,2024-05-08 21:58:34,0,606,1715216651,1715216654,2024-05-08 22:04:14,2024-05-08 22:04:11,0101000020E6100000598B4F0130A245C0B64AB0389CE9...,POINT (-43.26709 -22.91254)
926587,B25535,-22.91180,-43.26809,1715216344,2024-05-08 21:59:04,31,606,1715216651,1715216654,2024-05-08 22:04:14,2024-05-08 22:04:11,0101000020E61000003C31EBC550A245C0287E8CB96BE9...,POINT (-43.26809 -22.91180)
926588,B25535,-22.91180,-43.26809,1715216344,2024-05-08 21:59:04,31,606,1715216651,1715216674,2024-05-08 22:04:34,2024-05-08 22:04:11,0101000020E61000003C31EBC550A245C0287E8CB96BE9...,POINT (-43.26809 -22.91180)
926589,B25535,-22.91319,-43.26550,1715216284,2024-05-08 21:58:04,26,606,1715216658,1715216674,2024-05-08 22:04:34,2024-05-08 22:04:18,0101000020E6100000448B6CE7FBA145C0C408E1D1C6E9...,POINT (-43.26550 -22.91319)


In [15]:
routes

,linha,week_day,start_point,end_point,route_line_out,route_line_back
0,3,Segunda,POINT(-43.495039457092815 -22.883836915499124),POINT(-43.49507423162255 -22.884352563230294),LINESTRING(-43.495039457092815 -22.88383691549...,LINESTRING(-43.495039457092815 -22.88383691549...
1,774,Segunda,POINT(-43.32742040832666 -22.807352655457702),POINT(-43.341946615320566 -22.870417119067444),LINESTRING(-43.32742040832666 -22.807352655457...,LINESTRING(-43.341946615320566 -22.87041711906...
2,864,Segunda,POINT(-43.46365464290197 -22.876013326220658),POINT(-43.55592869637883 -22.901696375116064),LINESTRING(-43.46365464290197 -22.876013326220...,LINESTRING(-43.55592869637883 -22.901696375116...
3,852,Domingo,POINT(-43.64163358974359 -23.00088102564103),POINT(-43.554660550458706 -22.90174194954129),LINESTRING(-43.64163358974359 -23.000881025641...,LINESTRING(-43.554660550458706 -22.90174194954...
4,483,Segunda,POINT(-43.28434311311084 -22.836897800432855),POINT(-43.19716470131702 -22.98572288570085),LINESTRING(-43.28434311311084 -22.836897800432...,LINESTRING(-43.19716470131702 -22.985722885700...
...,...,...,...,...,...,...
151,759,Domingo,POINT(-43.656020895877376 -22.93565217230444),POINT(-43.34313948227349 -22.831867743387733),LINESTRING(-43.656020895877376 -22.93565217230...,LINESTRING(-43.34313948227349 -22.831867743387...
152,606,Domingo,POINT(-43.2110094648905 -22.89957320162565),POINT(-43.29415438547487 -22.895792050279326),LINESTRING(-43.2110094648905 -22.8995732016256...,LINESTRING(-43.29415438547487 -22.895792050279...
153,422,Domingo,POINT(-43.27246526623576 -22.911441492767004),POINT(-43.17939394942529 -22.931299379310346),LINESTRING(-43.27246526623576 -22.911441492767...,LINESTRING(-43.17939394942529 -22.931299379310...
154,100,Domingo,POINT(-43.19174866415094 -22.90452693584906),POINT(-43.19736202665013 -22.98472788968082),LINESTRING(-43.19174866415094 -22.904526935849...,LINESTRING(-43.19736202665013 -22.984727889680...


### Predict kilometers traveled on going trajectory
- Lets use 606 ate monday